In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import healpy as hp
from scipy.stats import binned_statistic
import rubin_sim.maf as maf
from rubin_sim.data import get_baseline
import time
import sys
import sqlite3
import json
import glob
from natsort import natsorted
import os

import imageio
from IPython.display import Image

In [2]:
baseline = "first_year_baseline_v3_3_10yrs_db_noDD_noTwi"
night_min = 0
night_max = 365
# keep_nights = [3,90,180,270,365]
keep_nights = [7,91,182,273,364]

# keep_nights = np.arange(night_min,night_max+1,1)


filt = "_r_and"
# filt = ""
# nside = 32
nside = 256
# tscale = 28
# tscale = 3
tscale = 7
# metric = "doAllTemplateMetrics_reduceCount"
metric = "CountMetric"

savedir = "{}_nside_{}_t-{}d_{}".format(baseline,nside,tscale,metric)

data_path = "/home/jrobinson/rubin_templates/remove_no_template_results_{}_{}".format(nside,baseline)

# metric_fname = "{}_tscale-{}_nside-{}_{}{}_night_lt_*_and_note_not_like_DD_and_note_not_like_twilight_HEAL.npz".format(
#                     baseline.split("_db_noDD_noTwi")[0],tscale,nside,metric,filt)
metric_fname = "{}_tscale-{}_nside-{}_{}{}_night_lt_*_and_note_not_like_DD_and_note_not_like_twilight_HEAL.npz".format(
                    baseline,tscale,nside,metric,filt)

savefig_path = "plot_template_coverage_figs/{}".format(savedir)

if not os.path.isdir(savefig_path):
    print("mkdir {}".format(savefig_path))
    os.mkdir(savefig_path)


mkdir plot_template_coverage_figs/first_year_baseline_v3_3_10yrs_db_noDD_noTwi_nside_256_t-7d_CountMetric


In [9]:
data_path

'/home/jrobinson/rubin_templates/remove_no_template_results_256_first_year_baseline_v3_3_10yrs_db_noDD_noTwi'

In [3]:
metric_files = glob.glob("{}/{}".format(data_path,metric_fname))
metric_files = natsorted(metric_files)

In [4]:
metric_files

[]

In [5]:
metric_files_keep = [x for x in metric_files if int(x.split("_lt_")[-1].split("_")[0]) in keep_nights]

In [6]:
metric_files_keep

[]

In [7]:
if len(metric_files_keep)<len(metric_files):
    savedir = "keep_nights_" + savedir
    
print(savedir)

first_year_baseline_v3_3_10yrs_db_noDD_noTwi_nside_256_t-7d_CountMetric


In [8]:
mb = maf.MetricBundle.load(metric_files[0])

IndexError: list index out of range

In [ ]:
mb.__dict__

In [ ]:
mb.file_root

In [ ]:
    
# # divide year 1 into chunks of a given template_timescale
# template_nights = np.arange(0,night_max+tscale,tscale)
# template_nights[-1] = night_max # consider only the first year
    
# template_nights

In [ ]:
# step = night_max/4.
# plot_nights = np.arange(step,night_max+step,step)
# step, plot_nights

In [ ]:
# arg_list = []
# for i in range(len(plot_nights)):
#     j = np.argmin(np.abs(template_nights - plot_nights[i]))
#     arg_list.append(j)
#     print(j, template_nights[j])
# print

In [ ]:
# retrieve data/mask for all the masked arrays
# each value in data corresponds to the metric value for some slice point
data = mb.metric_values.data
mask = mb.metric_values.mask
metric_vals = np.ma.array(data, mask=mask)
        
# replace the masked values with nan
metric_vals.fill_value = np.nan

In [ ]:
#plot the skymap
x = hp.mollview(metric_vals, title=mb.file_root, 
                cbar = None)
hp.graticule()

# customise the colorbar
fig = plt.gcf()
ax = plt.gca()
image = ax.get_images()[0]
cbar = fig.colorbar(image, ax=ax, orientation = "horizontal", shrink = 0.5, location = "bottom",
                   pad = 0.05)
cbar.set_label("count n visits")

#     fname = "{}.png".format("".join(title.split(" ")))
#     plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")

plt.show()

In [ ]:
len(metric_files)

In [ ]:
### set min/max to end of year 1 values?
_min = 0

mb = maf.MetricBundle.load(metric_files[-1])
data = mb.metric_values.data
mask = mb.metric_values.mask
metric_vals = np.ma.array(data, mask=mask)
    
_max = np.amax(metric_vals)

print(_min,_max)

In [ ]:
img_files = []

for i in range(len(metric_files)):

#     if metric_files[i] not in metric_files_keep:
#         continue

    mb = maf.MetricBundle.load(metric_files[i])
    data = mb.metric_values.data
    mask = mb.metric_values.mask
    metric_vals = np.ma.array(data, mask=mask)

    print(metric_vals,len(metric_vals))

    # replace the masked values with nan
    metric_vals.fill_value = np.nan
    
    metric_fname = mb.file_root
    print(metric_fname)
    
    title = metric_fname.split("nside-{}_".format(nside))[-1].split("_and_note")[0]
    t_split = title.split("_")
#     title = "$N$ visits, baseline, filter = ${}$, $t$ = {}d".format(t_split[1], t_split[-1])
    title = "$N$ visits, filter = ${}$, $t$ = {}d".format(t_split[1], t_split[-1])
    print(title)
    
    #plot the skymap
    x = hp.mollview(metric_vals, title=title, 
#                     min = _min, max = _max,
                    cbar = None)
    hp.graticule()

    # customise the colorbar
    fig = plt.gcf()
    ax = plt.gca()
    image = ax.get_images()[0]
    cbar = fig.colorbar(image, ax=ax, orientation = "horizontal", shrink = 0.5, location = "bottom",
                       pad = 0.05)
    cbar.set_label("count n visits")

    fname = "{}/{}.png".format(savefig_path,metric_fname)
    img_files.append(fname)
    plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")
    
    if len(metric_files_keep)<len(metric_files):
        fname = "{}/{}.pdf".format(savefig_path,metric_fname)
        plt.savefig(fname, facecolor="w", transparent=True, bbox_inches="tight")

    plt.show()

In [ ]:
# Build GIF
gif_file = '{}/{}.gif'.format(savefig_path,"_".join(metric_fname.split("_lt_*_")).split(".npz")[0])
print(gif_file)
with imageio.get_writer(gif_file, mode='I') as writer:
    for filename in img_files:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
display(Image(data=open(gif_file,'rb').read(), format='gif'))

In [ ]:
# Build mp4
mp4_file = '{}/{}.mp4'.format(savefig_path,"_".join(metric_fname.split("_lt_*_")).split(".npz")[0])
print(mp4_file)
with imageio.get_writer(mp4_file, mode='I') as writer:
    for filename in img_files:
        image = imageio.imread(filename)
        writer.append_data(image)